In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/HIV_inhibitors_classification_and_generation/research'

In [2]:
os.chdir("../")
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/HIV_inhibitors_classification_and_generation'

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from hivclass.utils.main_utils import plot_confusion_matrix
import random
import numpy as np
from pathlib import Path
from hivclass.utils.main_utils import *
# from rdkit import Chem
# import deepchem as dc

In [22]:
cnf = read_yaml(Path('params.yaml'))
cnf = cnf.BEST_PARAMETERS
cnf1 = read_yaml(Path('params.yaml'))
cnf1 = cnf1.BEST_PARAMETERS

In [17]:
cnf.HYPERPARAMETERS.early_stopping = 12

In [25]:
for folder in os.listdir('artifacts/data_transformation/test'):
    print(folder)

test_565.pt
test_534.pt
test_201.pt
test_444.pt
test_33.pt
test_544.pt
test_163.pt
test_194.pt
test_143.pt
test_98.pt
test_298.pt
test_196.pt
test_155.pt
test_244.pt
test_334.pt
test_275.pt
test_135.pt
test_61.pt
test_50.pt
test_417.pt
test_127.pt
test_435.pt
test_509.pt
test_447.pt
test_178.pt
test_108.pt
test_241.pt
test_530.pt
test_165.pt
test_346.pt
test_480.pt
test_238.pt
test_502.pt
test_219.pt
test_556.pt
test_521.pt
test_27.pt
test_471.pt
test_362.pt
test_577.pt
test_401.pt
test_338.pt
test_122.pt
test_515.pt
test_18.pt
test_8.pt
test_59.pt
test_459.pt
test_45.pt
test_277.pt
test_131.pt
test_317.pt
test_169.pt
test_167.pt
test_44.pt
test_214.pt
test_512.pt
test_274.pt
test_62.pt
test_91.pt
test_177.pt
test_488.pt
test_16.pt
test_232.pt
test_450.pt
test_243.pt
test_85.pt
test_28.pt
test_527.pt
test_443.pt
test_391.pt
test_374.pt
test_159.pt
test_304.pt
test_1.pt
test_501.pt
test_266.pt
test_302.pt
test_212.pt
test_11.pt
test_31.pt
test_554.pt
test_292.pt
test_172.pt
test_369.pt


In [24]:
if cnf == cnf1:
    print(True)
else:
    print(False)

True


In [4]:
preds =  [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1]
labels = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]

cm = confusion_matrix(labels, preds)
print(cm)

plot_confusion_matrix(cm, 'research', 0)

[[5 4]
 [3 7]]


In [4]:
hiv = pd.read_csv('artifacts/data_ingestion/HIV.csv')

In [6]:
hiv['HIV_active'].value_counts()

HIV_active
0    39684
1     1443
Name: count, dtype: int64

In [14]:
train_df, test_df = train_test_split(hiv, stratify=hiv['HIV_active'], test_size=0.2, random_state=42)

In [10]:
test_df['HIV_active'].value_counts()

HIV_active
0    7937
1     289
Name: count, dtype: int64

In [ ]:
def t_t_split_balance(df, split_size):
    # Separate positive and negative cases
    p_val = df[df.HIV_active == 1].to_numpy()
    n_val = df[df.HIV_active == 0].to_numpy()

    # Ensure class balance by selecting the smaller group as the target
    if len(p_val) >= len(n_val):
        big, small = p_val, n_val
    else:
        big, small = n_val, p_val

    # Stratified test split
    small_train, small_test = train_test_split(small, test_size=split_size, random_state=42)
    big_train, big_test = train_test_split(big, test_size=(split_size * len(small) / len(big)), random_state=42)

    test = np.concatenate([small_test, big_test])
    
    # Ensure the train set remains balanced by oversampling the smaller class
    train = np.concatenate([big_train, random.choices(small_train, k=len(big_train) - len(small_train))])

    # Convert back to DataFrame
    train_df = pd.DataFrame(train, columns=df.columns)
    test_df = pd.DataFrame(test, columns=df.columns)

    return train_df.sample(frac=1, random_state=42), test_df.sample(frac=1, random_state=42)

In [ ]:
train_df, test_df = t_t_split_balance(hiv, 0.2)

In [ ]:
train_df.head()

In [ ]:
p_list = p_df.values.tolist()

In [ ]:
test = random.choices(p_list, k=int(0.2*len(p_list)))
test

In [ ]:
test = pd.DataFrame(test)

In [ ]:
for row in hiv.head().iterrows():
    if hiv.eq(row[1]).all(axis=1).any():
        print(True)

In [ ]:
if list(test_df.columns) == ['smils', 'activity', 'HIV_active']:
    print(True)

In [ ]:
test_df['HIV_active'][0] = None

In [ ]:
bool(test_df['HIV_active'].isnull().sum())

In [4]:
train_pth = 'artifacts/data_transformation/train.csv'
test_pth = 'artifacts/data_transformation/test.csv'
train = pd.read_csv(train_pth)
test = pd.read_csv(test_pth)
# list(train.index)
# train.HIV_active.isna().sum().sum()


In [5]:
train.HIV_active.value_counts()

HIV_active
0    39388
1     1154
Name: count, dtype: int64

In [ ]:
import torch

mol = torch.load('artifacts/data_transformation/test/test_0.pt', weights_only=False)

In [ ]:
mol.y.item()

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train, test_size=0.2, random_state=42, stratify=train.HIV_active)

In [ ]:
val_df.HIV_active.value_counts()

In [ ]:
type(val_df.index.tolist()[0])

In [ ]:
name = [f'test_{i+1}' for i in train.index]
train.insert(0, 'name', name)
train.head()

In [ ]:
mol_obj = Chem.MolFromSmiles(train.smiles[1])

In [ ]:
for atom in mol_obj.GetAtoms():
    print(atom.GetSymbol(), atom.GetChiralTag())

In [ ]:
bonds = mol_obj.GetBonds()

for bond in bonds:
    print(bond.GetBeginAtom().GetAtomicNum())
    print(bond.GetIsConjugated())
    print(bond.GetEndAtom().GetAtomicNum())
    print('___')

In [ ]:
featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
f = featurizer._featurize(mol_obj)
# data = f.to_pyg_graph()
f.node_features

In [ ]:
datasets = {name: pd.read_csv(csv) for name, csv in 
                zip(["train", "test"], ['artifacts/data_transformation/train.csv', 'artifacts/data_transformation/test.csv'])}

In [ ]:
datasets

In [ ]:
from hivclass.utils.molecule_dataset import MoleculeDataset
import sys

In [ ]:
train_dataset = MoleculeDataset(
    'artifacts/data_ingestion',
    'artifacts/data_transformation',
    'HIV.csv',
    ['test', 'train', 'test.csv', 'train.csv']
)

In [ ]:
sample = train_dataset.get(0)

In [ ]:
train_dataset.len()

In [ ]:
non = 0
for i, sample in enumerate(train_dataset):
    sys.stdout.write(f'Count: {i}')
    sys.stdout.flush()
    print()
    
    if sample.y.item() == None:
        non += 1

print(non)

In [ ]:
train_dataset.get(15821)

In [ ]:
model_folder_path = 'artifacts/model_trainer/models/2'
model_name = os.listdir(model_folder_path)[0]
model_path = os.path.join(model_folder_path, model_name)

In [ ]:
model_path